<a href="https://colab.research.google.com/github/wrightj2/SEJ/blob/main/SEJ_Striking_Distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Striking Distance Report Creator v1
by [LeeFootSEO](https://twitter.com/LeeFootSEO) October 2021


In [103]:
!pip install pandas

In [104]:
import pandas as pd
from pandas import DataFrame, Series
from typing import Union
from google.colab import files

# Set Variables

In [105]:
# set all variables here
min_volume = 1  # set the minimum search volume
min_position = 1  # set the minimum position  / default = 4
max_position = 200 # set the maximum position  / default = 20
drop_all_true = True  # If all checks (h1/title/copy) are true, remove the recommendation (Nothing to do)
pagination_filters = "filterby|page|p="  # filter patterns used to detect and drop paginated pages

# Upload the Keyword Export File from Ahrefs or SEMrush

*   This file should be a report of all the keywords a site is ranking for.
*   Ahrefs (v1 and v2) & semrush keyword exports can be uploaded without modification.
*   .csv file in UTF-8 Format

For any other keyword exports, your csv file needs to contain the following column names:
```URL```
```Keyword```
```Volume```
```Position```

In [106]:
# upload the keyword export
upload = files.upload()
upload = list(upload.keys())[0]  # get the name of the uploaded file
df_keywords = pd.read_csv(
    (upload),
    error_bad_lines=False,
    low_memory=False,
    encoding="utf8",
    dtype={
        "URL": "str",
        "Keyword": "str",
        "Volume": "str",
        "Position": int,
        "Current URL": "str",
        "Search Volume": int,
    },
)

print("Uploaded Keyword CSV File Successfully!")
print(df_keywords)

Saving procopytips.com--organic-keywords-subdomains__2021-11-04_11-40-50.csv to procopytips.com--organic-keywords-subdomains__2021-11-04_11-40-50 (7).csv
Uploaded Keyword CSV File Successfully!
                     Keyword  ...         Current date
0              flys or flies  ...  2021-11-03 00:28:11
1          writing aesthetic  ...  2021-11-03 07:07:39
2                disfunction  ...  2021-11-03 20:03:05
3              flies or flys  ...  2021-11-03 19:50:06
4           skeleton outline  ...  2021-10-29 00:35:54
...                      ...  ...                  ...
1303                app sumo  ...  2021-11-03 15:56:06
1304  freelance writing gigs  ...                  NaN
1305          keywordshitter  ...  2021-10-22 09:31:50
1306                 appsumo  ...  2021-11-04 03:30:09
1307         yeses or yes\'s  ...  2021-10-08 11:19:54

[1308 rows x 17 columns]


# Upload the Crawl Export

If you'd like to check if keywords are in the copy (you should!) we recommend setting a custom extraction as shown in the image below:

1.   The extractor MUST be named 'Copy' as below
2.   'Extract Text' must be choosen from the drop down

![](https://drive.google.com/uc?export=view&id=16SVAm_k3QwYe9PuZGq3xywJ2smdeTMN0)

For any other crawler, your csv file needs to contain the following column names: `Address` `Title 1` `H1-1` `Copy 1` `Indexability` (Optional)


In [107]:
# upload the crawl export from Screaming Frog
upload = files.upload()
upload = list(upload.keys())[0]  # get the name of the uploaded file
df_crawl = pd.read_csv(
    (upload),
        error_bad_lines=False,
        low_memory=False,
        encoding="utf8",
        dtype="str",
    )

print("Uploaded Crawl Dataframe Successfully!")
print(df_crawl.head())

Saving internal_html - internal_html.csv to internal_html - internal_html (1).csv
Uploaded Crawl Dataframe Successfully!
                                             Address  ...      Crawl Timestamp
0                           https://procopytips.com/  ...  2021-11-04 12:57:06
1  https://procopytips.com/a-forest-copywriting-f...  ...  2021-11-04 12:57:07
2                      https://procopytips.com/blog/  ...  2021-11-04 12:57:07
3     https://procopytips.com/copywriting-resources/  ...  2021-11-04 12:57:08
4        https://procopytips.com/author/wpx_rebekah/  ...  2021-11-04 12:57:07

[5 rows x 58 columns]


# Clean the Keyword Dataframe

In [108]:
# standardise the column names between the different input files
df_keywords.rename(
    columns={
        "Current position": "Position",
        "Current URL": "URL",
        "Search Volume": "Volume",
        "Volume (desc)": "Volume",
        "Volume (asc)": "Volume",
    },
    inplace=True,
)

# keep only the following columns from the keyword dataframe
cols = "URL", "Keyword", "Volume", "Position"
df_keywords = df_keywords.reindex(columns=cols)

try:
    # clean the data. (v1 of the ahrefs keyword export combines strings and ints in the volume column)
    df_keywords["Volume"] = df_keywords["Volume"].str.replace("0-10", "0")
except AttributeError:
    pass

# clean the keyword data
df_keywords = df_keywords[df_keywords["URL"].notna()]  # remove any missing values
df_keywords = df_keywords[df_keywords["Volume"].notna()]  # remove any missing values
df_keywords = df_keywords.astype({"Volume": int})  # change data type to int
df_keywords = df_keywords.sort_values(by="Volume", ascending=False)  # sort by highest vol to keep the top opportunity

# make new dataframe to merge search volume back in later
df_keyword_vol = df_keywords[["Keyword", "Volume"]]

# drop rows if minimum search volume doesn't match specified criteria
try:
  df_keywords.loc[df_keywords["Volume"] < min_volume, "Volume_Too_Low"] = "drop"
  df_keywords = df_keywords[~df_keywords["Volume_Too_Low"].isin(["drop"])]
except ValueError:
  print("Please Check if 'Volume' Column is Named Correctly!")
  pass

# drop rows if minimum search position doesn't match specified criteria
df_keywords.loc[df_keywords["Position"] <= min_position, "Position_Too_High"] = "drop"
df_keywords = df_keywords[~df_keywords["Position_Too_High"].isin(["drop"])]

# drop rows if maximum search position doesn't match specified criteria
df_keywords.loc[df_keywords["Position"] >= max_position, "Position_Too_Low"] = "drop"
df_keywords = df_keywords[~df_keywords["Position_Too_Low"].isin(["drop"])]

print(df_keywords)

                                                    URL  ... Position_Too_Low
938              https://procopytips.com/flys-or-flies/  ...              NaN
1306            https://procopytips.com/appsumo-review/  ...              NaN
0                https://procopytips.com/flys-or-flies/  ...              NaN
254               https://procopytips.com/lets-vs-lets/  ...              NaN
456          https://procopytips.com/per-se-or-per-say/  ...              NaN
...                                                 ...  ...              ...
859     https://procopytips.com/freelance-writing-jobs/  ...              NaN
430     https://procopytips.com/freelance-writing-jobs/  ...              NaN
868     https://procopytips.com/freelance-writing-jobs/  ...              NaN
869             https://procopytips.com/appsumo-review/  ...              NaN
1041  https://procopytips.com/energize-creative-powers/  ...              NaN

[1098 rows x 7 columns]


# Clean the Crawl Dataframe

In [109]:
# keep only the following columns from the crawl dataframe
cols = "Address", "Indexability", "Title 1", "H1-1", "Copy 1"
df_crawl = df_crawl.reindex(columns=cols)

# drop non-indexable rows
df_crawl = df_crawl[~df_crawl["Indexability"].isin(["Non-Indexable"])]

# drop pagination
df_crawl = df_crawl[~df_crawl.Address.str.contains(pagination_filters)]

# standardise the column names
df_crawl.rename(columns={"Address": "URL", "Title 1": "Title", "H1-1": "H1", "Copy 1": "Copy"}, inplace=True)
df_crawl.head()

,URL,Indexability,Title,H1,Copy
0,https://procopytips.com/,Indexable,Home - ProCopyTips,Making Ordinary Copy Extraordinary,Making Ordinary Copy Extraordinary Whether you...
1,https://procopytips.com/a-forest-copywriting-f...,Indexable,The A FOREST Copywriting Formula - ProCopyTips,The A FOREST Copywriting Formula,The A FOREST copywriting formula is a mnemonic...
2,https://procopytips.com/blog/,Indexable,Blog - ProCopyTips,NaN,NaN
3,https://procopytips.com/copywriting-resources/,Indexable,Copywriting Resources - ProCopyTips,Copywriting Resources,We’ll keep adding to this page as we add more ...
4,https://procopytips.com/author/wpx_rebekah/,Indexable,Rebekah Villon - ProCopyTips,Rebekah Villon,NaN


# Group the Keywords

In [110]:
# groups the URLs (remove the dupes and combines stats)
# make a copy of the keywords dataframe for grouping - this ensures stats can be merged back in later from the OG df
df_keywords_group = df_keywords.copy()
df_keywords_group["KWs in Striking Dist."] = 1  # used to count the number of keywords in striking distance
df_keywords_group = (
    df_keywords_group.groupby("URL")
    .agg({"Volume": "sum", "KWs in Striking Dist.": "count"})
    .reset_index()
)
df_keywords_group.head()

,URL,Volume,KWs in Striking Dist.
0,https://procopytips.com/,2890,5
1,https://procopytips.com/3-whys/,110,2
2,https://procopytips.com/about/,30,1
3,https://procopytips.com/affect-change-or-effec...,12150,43
4,https://procopytips.com/aida-copywriting-formu...,1090,13


# Display in Adjacent Rows ala Grepwords Style


In [111]:
# create a new df, combine the merged data with the original data. display in adjacent rows ala grepwords
df_merged_all_kws = df_keywords_group.merge(
    df_keywords.groupby("URL")["Keyword"]
    .apply(lambda x: x.reset_index(drop=True))
    .unstack()
    .reset_index()
)
#print(df_merged_all_kws)
# sort by biggest opportunity
df_merged_all_kws = df_merged_all_kws.sort_values(
    by="KWs in Striking Dist.", ascending=False
)

# reindex the columns to keep just the top five keywords
cols = "URL", "Volume", "KWs in Striking Dist.", 0, 1, 2, 3, 4
df_merged_all_kws = df_merged_all_kws.reindex(columns=cols)

# create union and rename the columns
df_striking: Union[Series, DataFrame, None] = df_merged_all_kws.rename(
    columns={
        "Volume": "Striking Dist. Vol",
        0: "KW1",
        1: "KW2",
        2: "KW3",
        3: "KW4",
        4: "KW5",
    }
)

# merges striking distance df with crawl df to merge in the title, h1 and category description
df_striking = pd.merge(df_striking, df_crawl, on="URL", how="inner")

In [112]:
# set the final column order and merge the keyword data in
cols = [
    "URL",
    "Title",
    "H1",
    "Copy",
    "Striking Dist. Vol",
    "KWs in Striking Dist.",
    "KW1",
    "KW1 Vol",
    "KW1 in Title",
    "KW1 in H1",
    "KW1 in Copy",
    "KW2",
    "KW2 Vol",
    "KW2 in Title",
    "KW2 in H1",
    "KW2 in Copy",
    "KW3",
    "KW3 Vol",
    "KW3 in Title",
    "KW3 in H1",
    "KW3 in Copy",
    "KW4",
    "KW4 Vol",
    "KW4 in Title",
    "KW4 in H1",
    "KW4 in Copy",
    "KW5",
    "KW5 Vol",
    "KW5 in Title",
    "KW5 in H1",
    "KW5 in Copy",
]
# re-index the columns to place them in a logical order + inserts new blank columns for kw checks.
df_striking = df_striking.reindex(columns=cols)

In [113]:
#merge in keyword data for each keyword column (KW1 - KW5)
df_striking = pd.merge(df_striking, df_keyword_vol, left_on="KW1", right_on="Keyword", how="left")
df_striking['KW1 Vol'] = df_striking['Volume']
df_striking.drop(['Keyword', 'Volume'], axis=1, inplace=True)
df_striking = pd.merge(df_striking, df_keyword_vol, left_on="KW2", right_on="Keyword", how="left")
df_striking['KW2 Vol'] = df_striking['Volume']
df_striking.drop(['Keyword', 'Volume'], axis=1, inplace=True)
df_striking = pd.merge(df_striking, df_keyword_vol, left_on="KW3", right_on="Keyword", how="left")
df_striking['KW3 Vol'] = df_striking['Volume']
df_striking.drop(['Keyword', 'Volume'], axis=1, inplace=True)
df_striking = pd.merge(df_striking, df_keyword_vol, left_on="KW4", right_on="Keyword", how="left")
df_striking['KW4 Vol'] = df_striking['Volume']
df_striking.drop(['Keyword', 'Volume'], axis=1, inplace=True)
df_striking = pd.merge(df_striking, df_keyword_vol, left_on="KW5", right_on="Keyword", how="left")
df_striking['KW5 Vol'] = df_striking['Volume']
df_striking.drop(['Keyword', 'Volume'], axis=1, inplace=True)


In [114]:
# drop duplciate url rows
df_striking.drop_duplicates(subset="URL", inplace=True) 

# replace nan values with empty strings
df_striking = df_striking.fillna("")

# drop the title, h1 and category description to lower case so kws can be matched to them
df_striking["Title"] = df_striking["Title"].str.lower()
df_striking["H1"] = df_striking["H1"].str.lower()
df_striking["Copy"] = df_striking["Copy"].str.lower()

In [115]:
# check whether a keyword appears in title, h1 or category description
df_striking["KW1 in Title"] = df_striking.apply(lambda row: row["KW1"] in row["Title"], axis=1)
df_striking["KW1 in H1"] = df_striking.apply(lambda row: row["KW1"] in row["H1"], axis=1)
df_striking["KW1 in Copy"] = df_striking.apply(lambda row: row["KW1"] in row["Copy"], axis=1)
df_striking["KW2 in Title"] = df_striking.apply(lambda row: row["KW2"] in row["Title"], axis=1)
df_striking["KW2 in H1"] = df_striking.apply(lambda row: row["KW2"] in row["H1"], axis=1)
df_striking["KW2 in Copy"] = df_striking.apply(lambda row: row["KW2"] in row["Copy"], axis=1)
df_striking["KW3 in Title"] = df_striking.apply(lambda row: row["KW3"] in row["Title"], axis=1)
df_striking["KW3 in H1"] = df_striking.apply(lambda row: row["KW3"] in row["H1"], axis=1)
df_striking["KW3 in Copy"] = df_striking.apply(lambda row: row["KW3"] in row["Copy"], axis=1)
df_striking["KW4 in Title"] = df_striking.apply(lambda row: row["KW4"] in row["Title"], axis=1)
df_striking["KW4 in H1"] = df_striking.apply(lambda row: row["KW4"] in row["H1"], axis=1)
df_striking["KW4 in Copy"] = df_striking.apply(lambda row: row["KW4"] in row["Copy"], axis=1)
df_striking["KW5 in Title"] = df_striking.apply(lambda row: row["KW5"] in row["Title"], axis=1)
df_striking["KW5 in H1"] = df_striking.apply(lambda row: row["KW5"] in row["H1"], axis=1)
df_striking["KW5 in Copy"] = df_striking.apply(lambda row: row["KW5"] in row["Copy"], axis=1)

# delete true / false values if there is no keyword
df_striking.loc[df_striking["KW1"] == "", ["KW1 in Title", "KW1 in H1", "KW1 in Copy"]] = ""
df_striking.loc[df_striking["KW2"] == "", ["KW2 in Title", "KW2 in H1", "KW2 in Copy"]] = ""
df_striking.loc[df_striking["KW3"] == "", ["KW3 in Title", "KW3 in H1", "KW3 in Copy"]] = ""
df_striking.loc[df_striking["KW4"] == "", ["KW4 in Title", "KW4 in H1", "KW4 in Copy"]] = ""
df_striking.loc[df_striking["KW5"] == "", ["KW5 in Title", "KW5 in H1", "KW5 in Copy"]] = ""
df_striking.head()

,URL,Title,H1,Copy,Striking Dist. Vol,KWs in Striking Dist.,KW1,KW1 Vol,KW1 in Title,KW1 in H1,KW1 in Copy,KW2,KW2 Vol,KW2 in Title,KW2 in H1,KW2 in Copy,KW3,KW3 Vol,KW3 in Title,KW3 in H1,KW3 in Copy,KW4,KW4 Vol,KW4 in Title,KW4 in H1,KW4 in Copy,KW5,KW5 Vol,KW5 in Title,KW5 in H1,KW5 in Copy
0,https://procopytips.com/freelance-writing-jobs/,awesome places to find paid freelance writing ...,awesome places to find paid freelance writing ...,it can be tough and time consuming to find pai...,20590,258,freelance writer jobs,2300,False,False,False,freelance writing jobs for beginners,2000,False,False,False,writing jobs online,1100,False,False,True,freelance writing jobs online,1000,False,False,True,paid online writing jobs,450,False,False,False
1,https://procopytips.com/tv-commercial-script/,how to write a 30-second tv commercial script ...,how to write a 30-second tv commercial script,"with a tv commercial script, you have to make ...",6900,94,commercial script example,350,False,False,False,30 second commercials scripts,300,False,False,False,tv commercial scripts,300,False,False,False,ad script,250,False,False,True,30 sec commercial script,250,False,False,False
2,https://procopytips.com/freelance-copywriting-...,freelance copywriting fees - what and how to c...,freelance copywriting fees – what and how to c...,if you’re struggling to set the correct freela...,3680,69,freelance copywriting,450,True,True,True,what is freelance copywriting,200,False,False,False,freelance writing rates,200,False,False,True,copywriter hourly rate,150,False,False,False,freelance writer rates,100,False,False,False
3,https://procopytips.com/flys-or-flies/,flys or flies - what’s the difference - procop...,flys or flies – what’s the difference,using flys or flies incorrectly is a common gr...,44370,58,flys,21000,True,True,True,flys or flies,10000,True,True,True,flies or flys,5100,False,False,False,fly vs fly,2000,False,False,False,flies vs flys,700,False,False,False
4,https://procopytips.com/affect-change-or-effec...,affect change or effect change – which is corr...,affect change or effect change – which is corr...,the short answer as to whether to use affect c...,12150,43,effect change,3200,True,True,True,affect change,1800,True,True,True,affect change or effect change,1800,True,True,True,effect or affect change,1200,False,False,False,effect change or affect change,600,False,False,False


In [116]:
# drops rows if all values evaluate to true. (nothing for the user to do).
def true_dropper(col1, col2, col3):
    drop = df_striking.drop(
        df_striking[
            (df_striking[col1] == True)
            & (df_striking[col2] == True)
            & (df_striking[col3] == True)
        ].index
    )
    return drop

if drop_all_true == True:
    df_striking = true_dropper("KW1 in Title", "KW1 in H1", "KW1 in Copy")
    df_striking = true_dropper("KW2 in Title", "KW2 in H1", "KW2 in Copy")
    df_striking = true_dropper("KW3 in Title", "KW3 in H1", "KW3 in Copy")
    df_striking = true_dropper("KW4 in Title", "KW4 in H1", "KW4 in Copy")
    df_striking = true_dropper("KW5 in Title", "KW5 in H1", "KW5 in Copy")

In [117]:
df_striking.to_csv('Keywords in Striking Distance.csv', index=False)
files.download("Keywords in Striking Distance.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>